In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# @title Install nonstandard libraries
%%capture
!pip install ipywidgets
!pip install vtk
!pip install meshio
!pip install pyvista

import os, re

def replace_in_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Replace 'ufl' with 'ufl_legacy'
    content = re.sub(r'\bufl\b', 'ufl_legacy', content)

    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content)

def process_directory(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.py'):
                file_path = os.path.join(root, file)
                replace_in_file(file_path)

# dolfin
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"

# block
try:
    import block
except ImportError:
    !git clone "https://bitbucket.org/fenics-apps/cbc.block/src/master/"
    !pip install master/

# fenics_ii
try:
    import xii
except ImportError:
    !git clone "https://github.com/MiroK/fenics_ii"
    process_directory("fenics_ii/")
    !pip install fenics_ii/

# graphnics
try:
    import graphnics
except ImportError:
    !git clone "https://github.com/IngeborgGjerde/graphnics"
    !pip install graphnics/

In [3]:
WD_PATH = "/content/drive/MyDrive/Research/3D-1D"

import sys, os
sys.path.append(os.path.join(WD_PATH, 'modules'))
import visualizer
import coupler_sensitivity
import meshio
import scipy
import copy
import vtk
import json
import numpy as np
import matplotlib.pyplot as plt
import datetime
import networkx as nx
from matplotlib.gridspec import GridSpec
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import cKDTree
import importlib
from dolfin import *
from vtk.util.numpy_support import vtk_to_numpy
from xii import *
from graphnics import *

/usr/local/lib/python3.10/dist-packages/block/__init__.py:15: UserWarning: The cbc.block repository has moved to https://github.com/blocknics/cbc.block
  warnings.warn('The cbc.block repository has moved to https://github.com/blocknics/cbc.block', UserWarning)


Missing HsMG for fract norm computing


In [4]:
# @title Define G = serena data graph
%%capture

G = FenicsGraph()
ind = 0
branch_points = {}

for n in range(29):
    file_path = os.path.join(WD_PATH, 'data', 'pv_json1', f'Centerline_{str(n)}.mrk.json')
    f = open(file_path)
    data = json.load(f)
    f.close()

    # get coords + radius at each point
    points = data['markups'][0]['controlPoints']
    radius = data['markups'][0]['measurements'][3]['controlPointValues']
    G.add_nodes_from(range(ind - n, ind + len(points) - n))

    # check if first coord is branch point from previous centerlines
    v1 = 0
    for key, val in branch_points.items():
        if points[0]['position'] == val:
            v1 = key
            break

    # add coords and radius to nodes fenics graph
    v2 = ind - n + 1
    pos_v1 = points[0]['position']
    pos_v2 = points[1]['position']
    G.nodes[v1]["pos"] = pos_v1
    G.nodes[v2]["pos"] = pos_v2
    G.nodes[v1]["radius"] = radius[0]
    G.nodes[v2]["radius"] = radius[1]
    # add edge to fenics graph
    G.add_edge(v1, v2)

    for i in range(len(points)-2):
        v1 = ind - n + 1 + i
        v2 = v1 + 1
        # convert coordinates from mm to meters (divide by 1000)
        pos_v1 = [coord / 1000 for coord in points[0]['position']]
        pos_v2 = [coord / 1000 for coord in points[1]['position']]
        pos_v1 = points[i + 1]['position']
        pos_v2 = points[i + 2]['position']
        G.nodes[v1]["pos"] = pos_v1
        G.nodes[v2]["pos"] = pos_v2
        G.nodes[v1]["radius"] = radius[i + 1]
        G.nodes[v2]["radius"] = radius[i + 2]
        G.add_edge(v1, v2, radius=radius[i+1])


    # store last point as a branch point
    ind += len(points)
    branch_points.update({ind-n-1: pos_v2})

In [ ]:
# @title Define G = .vtk domain read
%%capture

# function to read .vtk file and extract information
def read_vtk(file_path):
    reader = vtk.vtkPolyDataReader()
    reader.SetFileName(file_path)
    reader.Update()
    output = reader.GetOutput()
    G = FenicsGraph()

    for i in range(output.GetNumberOfPoints()):
        point = output.GetPoint(i)
        damage_value = output.GetPointData().GetArray("Damage").GetValue(i)
        radius_value = output.GetPointData().GetArray("Radius").GetValue(i)
        G.add_node(i, pos=tuple(point), radius=radius_value, damage=damage_value)

    for i in range(output.GetNumberOfCells()):
        cell = output.GetCell(i)
        for j in range(cell.GetNumberOfPoints() - 1):
            G.add_edge(cell.GetPointId(j), cell.GetPointId(j + 1))

    return G

# file_path = WD_PATH + '/data/vtk/sortedDomain.vtk'
file_path = WD_PATH + '/oncopigReferenceData/ZPAF23S018/20230531/vesselNetwork_upDated.vtk'
G = read_vtk(file_path)

In [ ]:
# @title Clean Up Graph (OPTIONAL)
def cleanup_graph(G):
    G.remove_nodes_from(list(nx.isolates(G)))
    G = nx.convert_node_labels_to_integers(G)

    # Merge close nodes
    positions = nx.get_node_attributes(G, 'pos')
    positions_array = np.array(list(positions.values()))
    tree = cKDTree(positions_array)
    merged_nodes = set()

    for node, pos in positions.items():
        # Find nearby nodes within the threshold distance
        nearby_nodes = tree.query_ball_point(pos, 1.0e-4)

        # Merge with the first nearby node (if any) that hasn't been merged yet
        for other_node in nearby_nodes:
            if other_node != node and other_node not in merged_nodes:
                G = nx.contracted_nodes(G, node, other_node)
                merged_nodes.add(other_node)
                break  # Stop after merging with one node

    # Set zero radii to 0.1
    for node in G.nodes():
        if G.nodes[node]['radius'] == 0:
            G.nodes[node]['radius'] = 0.1

    return G

G = cleanup_graph(G)

In [99]:
importlib.reload(FEM_sensitivity)
test = FEM_sensitivity.FEMSensitivity(G)

Averaging over 792 cells: 100%|██████████| 792/792 [00:00<00:00, 830.95it/s]


In [101]:
importlib.reload(coupler_sensitivity)

results_directory = os.path.join(WD_PATH, "perfusion_results")
os.makedirs(results_directory, exist_ok=True)

k = 9.44e-3
mu = 4.5e-3 #Pa s
perf3 = k / mu
perf1 = 1.0 * perf3

output_files = coupler_sensitivity.run_perfusion_univ(G, directory_path=results_directory, perf3=perf3, perf1 = perf1, kappa_value=3.09e-5)

Averaging over 792 cells: 100%|██████████| 792/792 [00:03<00:00, 223.69it/s]


Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling form ffc_form_a225c20b77ff15a97ff415e95b4dd52831db2f47

INFO:FFC:Compiler stage 1: Analyzing form(s)
INFO:FFC:-----------------------------------
DEBUG:FFC:  Preprocessing form using 'uflacs' representation family.
INFO:FFC:  
INFO:FFC:  Geometric dimension:       3
  Number of cell subdomains: 0
  Rank:                      2
  Arguments:                 '(v_0, v_1)'
  Number of coefficients:    0
  Coefficients:              '[]'
  Unique elements:           'CG1(?,?), Vector<3 x CG1(?,?)>'
  Unique sub elements:       'CG1(?,?), Vector<3 x CG1(?,?)>'
  
INFO:FFC:  representation:    auto --> uflacs
INFO:FFC:  quadrature_rule:   auto --> default
INFO:FFC:  quadrature_degree: auto --> 0
INFO:FFC:  quadrature_degree: 0
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.141582 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:------------------------------

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling form ffc_form_d17bece013f66b2c1aefe1f28792805926c772ba

INFO:FFC:Compiler stage 1: Analyzing form(s)
INFO:FFC:-----------------------------------
DEBUG:FFC:  Preprocessing form using 'uflacs' representation family.
INFO:UFL_LEGACY:Adjusting missing element cell to interval3D.
INFO:UFL_LEGACY:Adjusting missing element cell to interval3D.
INFO:UFL_LEGACY:Adjusting missing element cell to interval3D.
INFO:FFC:  
INFO:FFC:  Geometric dimension:                 3
  Number of cell subdomains:           0
  Number of exterior_facet subdomains: 0
  Rank:                                2
  Arguments:                           '(v_0, v_1)'
  Number of coefficients:              3
  Coefficients:                        '[f_6735, f_6736, f_6743]'
  Unique elements:                     'CG1(?,?), R0(?,?), CG2(?,?), Vector<3 x CG1(
                                       ?,?)>'
  Unique sub elements:    

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling form ffc_form_de0ee8acc6ddcee13573272c110259884e436ce3

INFO:FFC:Compiler stage 1: Analyzing form(s)
INFO:FFC:-----------------------------------
DEBUG:FFC:  Preprocessing form using 'uflacs' representation family.
INFO:UFL_LEGACY:Adjusting missing element cell to interval3D.
INFO:UFL_LEGACY:Adjusting missing element cell to interval3D.
INFO:FFC:  
INFO:FFC:  Geometric dimension:       3
  Number of cell subdomains: 0
  Rank:                      2
  Arguments:                 '(v_0, v_1)'
  Number of coefficients:    2
  Coefficients:              '[f_6735, f_6743]'
  Unique elements:           'CG1(?,?), R0(?,?), CG2(?,?), Vector<3 x CG1(?,?)>'
  Unique sub elements:       'CG1(?,?), R0(?,?), CG2(?,?), Vector<3 x CG1(?,?)>'
  
INFO:FFC:  representation:    auto --> uflacs
INFO:FFC:  quadrature_rule:   auto --> default
INFO:FFC:  quadrature_degree: auto --> 4
INFO:FFC:  quadrature_degree:

In [100]:
test.save_vtk("test_results_2")